<a href="https://colab.research.google.com/github/akrem008/Data_glacier_internship/blob/main/Week6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset source: 15 Million Chess Games from Lichess (2013-2014) from Kaggle

We are going to read and manipulate a large CSV file with several different methods and determine computational efficiency. We will be using pandas, Modin, Ray, and Dask.

In [1]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0


Writing testutility.py


Writing YAML File


In [2]:
%%writefile file.yaml
file_type: csv
dataset_name: Lichess_2013_2014_Complete
file_name: Lichess_2013_2014_Complete
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - WhiteElo
    - BlackElo
    - WhiteName
    - BlackName
    - Winner
    - Termination
    - Site
    - Day
    - Month
    - Year
    - InitialTime
    - Increment
    - TimeControl
    - Opening
    - ECO
    - Number_of_Moves

Writing file.yaml


In [3]:
# Read config file
import testutility as util
config_data = util.read_config_file("file.yaml")

In [4]:
config_data

{'file_type': 'csv',
 'dataset_name': 'Lichess_2013_2014_Complete',
 'file_name': 'Lichess_2013_2014_Complete',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['WhiteElo',
  'BlackElo',
  'WhiteName',
  'BlackName',
  'Winner',
  'Termination',
  'Site',
  'Day',
  'Month',
  'Year',
  'InitialTime',
  'Increment',
  'TimeControl',
  'Opening',
  'ECO',
  'Number_of_Moves']}

In [6]:
# Normal reading process of the file
import pandas as pd
df_sample = pd.read_csv("Lichess_2013_2014_Complete.csv",delimiter=',')
df_sample.head()

,WhiteElo,BlackElo,WhiteName,BlackName,Winner,Termination,Site,Day,Month,Year,InitialTime,Increment,TimeControl,Opening,ECO,Number_of_Moves
0,1639,1403,BFG9k,mamalak,White,Normal,https://lichess.org/j1dkb5dw,31,12,2012,600,8,Rapid,French Defense: Normal Variation,C00,13.0
1,1654,1919,Desmond_Wilson,savinka59,White,Normal,https://lichess.org/a9tcp02g,31,12,2012,480,2,Rapid,"Queen's Pawn Game: Colle System, Anti-Colle",D04,18.0
2,1643,1747,Kozakmamay007,VanillaShamanilla,White,Normal,https://lichess.org/szom2tog,31,12,2012,420,17,Rapid,Four Knights Game: Italian Variation,C50,11.0
3,1824,1973,Naitero_Nagasaki,800,Black,Normal,https://lichess.org/rklpc7mk,31,12,2012,60,1,Bullet,Caro-Kann Defense: Goldman Variation,B12,47.0
4,1765,1815,nichiren1967,Naitero_Nagasaki,Black,Normal,https://lichess.org/1xb3os63,31,12,2012,60,1,Bullet,French Defense: La Bourdonnais Variation,C00,23.0


Pandas readcsv completed: 33s

In [7]:
# read the file using config file
file_type = config_data['file_type']
source_file = "./" + config_data['file_name'] + f'.{file_type}'
#print("",source_file)
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


,WhiteElo,BlackElo,WhiteName,BlackName,Winner,Termination,Site,Day,Month,Year,InitialTime,Increment,TimeControl,Opening,ECO,Number_of_Moves
0,1639,1403,BFG9k,mamalak,White,Normal,https://lichess.org/j1dkb5dw,31.0,12.0,2012.0,600.0,8.0,Rapid,French Defense: Normal Variation,C00,13.0
1,1654,1919,Desmond_Wilson,savinka59,White,Normal,https://lichess.org/a9tcp02g,31.0,12.0,2012.0,480.0,2.0,Rapid,"Queen's Pawn Game: Colle System, Anti-Colle",D04,18.0
2,1643,1747,Kozakmamay007,VanillaShamanilla,White,Normal,https://lichess.org/szom2tog,31.0,12.0,2012.0,420.0,17.0,Rapid,Four Knights Game: Italian Variation,C50,11.0
3,1824,1973,Naitero_Nagasaki,800,Black,Normal,https://lichess.org/rklpc7mk,31.0,12.0,2012.0,60.0,1.0,Bullet,Caro-Kann Defense: Goldman Variation,B12,47.0
4,1765,1815,nichiren1967,Naitero_Nagasaki,Black,Normal,https://lichess.org/1xb3os63,31.0,12.0,2012.0,60.0,1.0,Bullet,French Defense: La Bourdonnais Variation,C00,23.0


Reading File with YAML Config: 29s

In [8]:
#validate the header of the file
util.col_header_val(df,config_data)

column name and column length validation passed


1

Basic Validation Confirmed. All columns were converted to lower case and special characters were handled.

In [9]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['whiteelo', 'blackelo', 'whitename', 'blackname', 'winner',
       'termination', 'site', 'day', 'month', 'year', 'initialtime',
       'increment', 'timecontrol', 'opening', 'eco', 'number_of_moves'],
      dtype='object')
columns of YAML are: ['WhiteElo', 'BlackElo', 'WhiteName', 'BlackName', 'Winner', 'Termination', 'Site', 'Day', 'Month', 'Year', 'InitialTime', 'Increment', 'TimeControl', 'Opening', 'ECO', 'Number_of_Moves']


Converting File with pipeline delimiter

In [10]:
import csv

with open('Lichess_2013_2014_Complete.csv') as fin:
    # newline='' prevents extra newlines when using Python 3 on Windows
    with open('OutputFile.txt', 'w', newline='') as fout:
        reader = csv.DictReader(fin, delimiter=',')
        writer = csv.DictWriter(fout, reader.fieldnames, delimiter='|')
        writer.writeheader()
        writer.writerows(reader)

Time to convert to '|' delimiter: 2 mins 55 s



Convert output.txt to gz format

In [11]:
import gzip

with open("OutputFile.txt", 'rb') as orig_file:
    with gzip.open("OutputFile.txt.gz", 'wb') as zipped_file:
        zipped_file.writelines(orig_file)

Converting to gz format: 2m 26s

Now we will try reading file with other methods and see computational efficiency.

In [12]:
#Using Dask
!python -m pip install "dask[complete]"
import dask
import dask.dataframe as dd 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
df = dd.read_csv('Lichess_2013_2014_Complete.csv')
df.head()

,WhiteElo,BlackElo,WhiteName,BlackName,Winner,Termination,Site,Day,Month,Year,InitialTime,Increment,TimeControl,Opening,ECO,Number_of_Moves
0,1639,1403,BFG9k,mamalak,White,Normal,https://lichess.org/j1dkb5dw,31,12,2012,600,8,Rapid,French Defense: Normal Variation,C00,13
1,1654,1919,Desmond_Wilson,savinka59,White,Normal,https://lichess.org/a9tcp02g,31,12,2012,480,2,Rapid,"Queen's Pawn Game: Colle System, Anti-Colle",D04,18
2,1643,1747,Kozakmamay007,VanillaShamanilla,White,Normal,https://lichess.org/szom2tog,31,12,2012,420,17,Rapid,Four Knights Game: Italian Variation,C50,11
3,1824,1973,Naitero_Nagasaki,800,Black,Normal,https://lichess.org/rklpc7mk,31,12,2012,60,1,Bullet,Caro-Kann Defense: Goldman Variation,B12,47
4,1765,1815,nichiren1967,Naitero_Nagasaki,Black,Normal,https://lichess.org/1xb3os63,31,12,2012,60,1,Bullet,French Defense: La Bourdonnais Variation,C00,23


Dask read csv completed: 2s

In [14]:
!python -m pip install "ray"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59.4 MB 65 kB/s 
     |████████████████████████████████| 8.8 MB 31.8 MB/s 
     |████████████████████████████████| 4.1 MB 39.0 MB/s 
     |████████████████████████████████| 468 kB 55.5 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.47.0
    Uninstalling grpcio-1.47.0:
      Successfully uninstalled grpcio-1.47.0


In [25]:
import ray
ds = ray.data.read_csv("Lichess_2013_2014_Complete.csv")
ds.head()

(_execute_read_task pid=328) 2022-09-12 13:26:55,637	INFO worker.py:756 -- Task failed with retryable exception: TaskID(71b133a11e1c461cffffffffffffffffffffffff01000000).
(_execute_read_task pid=328) Traceback (most recent call last):
(_execute_read_task pid=328)   File "python/ray/_raylet.pyx", line 662, in ray._raylet.execute_task
(_execute_read_task pid=328)   File "python/ray/_raylet.pyx", line 666, in ray._raylet.execute_task
(_execute_read_task pid=328)   File "/usr/local/lib/python3.7/dist-packages/ray/data/_internal/lazy_block_list.py", line 576, in _execute_read_task
(_execute_read_task pid=328)     block = task()
(_execute_read_task pid=328)   File "/usr/local/lib/python3.7/dist-packages/ray/data/datasource/datasource.py", line 202, in __call__
(_execute_read_task pid=328)     for block in result:
(_execute_read_task pid=328)   File "/usr/local/lib/python3.7/dist-packages/ray/data/datasource/file_based_datasource.py", line 427, in read_files
(_execute_read_task pid=328)     f

RayTaskError(ArrowInvalid): ignored

Ray read csv: 1m 25s

#Summary

In [16]:
num_rows = len(df_sample.index)
num_cols = len(df_sample.columns)
file_size = '2GB'

time_pandas = '33s'
time_wYAML = '29s'
time_ray = '1min 25'
time_dask = '2s'


In [17]:
print(f'Total number of rows: {num_rows}')
print(f'Total number of columns: {num_cols}')
print(f'File Size: {file_size}')
print(f'Time for pandas read_csv: {time_pandas}')
print(f'Time for read_csv with YAML config: {time_wYAML}')
print(f'Time for ray read_csv: {time_ray}')
print(f'Time for dask read_csv: {time_dask}')

Total number of rows: 71333
Total number of columns: 16
File Size: 2GB
Time for pandas read_csv: 33s
Time for read_csv with YAML config: 29s
Time for ray read_csv: 1min 25
Time for dask read_csv: 2s
